In [ ]:
import scanpy as sc

In [ ]:
microglia_adata = sc.read_h5ad("rds/microglia_adata.h5ad")

In [ ]:

# 按照 group 分组，比较 AD vs WT
sc.tl.rank_genes_groups(
    microglia_adata,
    groupby='group',
    reference='WT',       # 用 WT 作为对照
    groups=['AD'],        # 要比较的组
    method='wilcoxon'     # 非参数检验
)

# 从结果中取出 Mertk
rg = microglia_adata.uns['rank_genes_groups']
# 名称列表、pvals、logfoldchanges 都是按 rank 排序的 numpy 数组
names = rg['names']['AD']
pvals = rg['pvals']['AD']
lfcs  = rg['logfoldchanges']['AD']

# 找到 Mertk 的索引
idx = list(names).index('Mertk')
print(f"Mertk vs WT: log2FC = {lfcs[idx]:.3f}, p-value = {pvals[idx]:.3e}")


In [ ]:
# 可视化结果
sc.pl.rank_genes_groups_dotplot(
		microglia_adata,
		groupby='group',
		var_names=['Mertk'],
		title='Mertk Expression in AD vs WT',
		use_raw=False,
		cmap='viridis',
		standard_scale='var'
)

In [ ]:
# 做散点图/小提琴图
sc.pl.rank_genes_groups_violin(
		microglia_adata,
		gene_names=['Mertk'],
		jitter=0.4,  # 添加抖动
		split=True,  # 按照 group 分组
		use_raw=False
)

In [ ]:
sc.pl.violin(microglia_adata, ["Mertk"], groupby="group")

In [ ]:
# 将microglia_adata 's obs 中的 group 列转换为分类变量, meanwhile make WT before AD
microglia_adata.obs['group'] = microglia_adata.obs['group'].astype('category')
microglia_adata.obs['group'] = microglia_adata.obs['group'].cat.reorder_categories(['WT', 'AD'], ordered=True)


In [ ]:
sc.pl.violin(microglia_adata, ["Mertk"], groupby="group")

In [ ]:
# 将图片保存为PDF
sc.pl.violin(
		microglia_adata, 
		["Mertk"], 
		groupby="group", 
		save="_mertk_violin.pdf"
)

In [ ]:
# 比较AD vs WT的DEG，然后做富集分析
sc.tl.rank_genes_groups(
		microglia_adata,
		groupby='group',
		reference='WT',       # 用 WT 作为对照
		groups=['AD'],        # 要比较的组
		method='wilcoxon'     # 非参数检验
)

In [ ]:
# 获取差异基因结果
de_genes = sc.get.rank_genes_groups_df(microglia_adata, group='AD', pval_cutoff= 0.2)
# 保存差异基因结果到CSV
de_genes.to_csv("mertk_ad_vs_wt_de_genes.csv", index=False)

In [ ]:
# ! pip install gprofiler-official

In [ ]:
de_genes['names'].to_list()  # 获取基因名称列表

In [ ]:
sc.queries.enrich(de_genes['names'].to_list(), org='mmusculus')

In [ ]:
res_go = sc.queries.enrich(de_genes['names'].to_list(), org='mmusculus')

In [ ]:
print(res_go.head())

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

In [ ]:
# 1. 数据预处理
# 计算富集因子和-log10(p值)
res_go['enrichment_factor'] = res_go['intersection_size'] / res_go['term_size']
res_go['-log10(p)'] = -np.log10(res_go['p_value'])

In [ ]:
# 创建简短的术语名称 (保留核心部分)
res_go['term_short'] = res_go['name'].apply(
    lambda x: x[:30] + '...' if len(x) > 33 else x
)


In [ ]:
res_go

In [ ]:
# 去掉res_go last row
res_go = res_go[:-1]

In [ ]:
# 添加类别标签
res_go['category'] = res_go['source'].map({
    'GO:BP': 'Biological Process',
    'GO:MF': 'Molecular Function',
    'GO:CC': 'Cellular Component'
})

In [ ]:
res_go

In [ ]:
# 筛选显著结果
sig_go = res_go[res_go['significant']].sort_values('p_value')


In [ ]:
# 2. 气泡图 - 最推荐的富集结果可视化方式
plt.figure(figsize=(12, 10))

# 创建气泡图
scatter = plt.scatter(
    x=sig_go['enrichment_factor'],
    y=sig_go['term_short'],
    s=sig_go['intersection_size'] * 20,  # 气泡大小表示重叠基因数量
    c=sig_go['-log10(p)'],               # 颜色表示显著性
    cmap='viridis',
    alpha=0.8,
    edgecolors='grey'
)


In [ ]:
# 3. 按类别分面条形图
plt.figure(figsize=(14, 10))
sns.set_theme(style="whitegrid")

# 创建分面图
g = sns.FacetGrid(
    sig_go,
    col='category',
    col_wrap=3,
    height=6,
    aspect=1.2,
    sharey=False
)

# 绘制条形图
g.map_dataframe(
    sns.barplot,
    x='enrichment_factor',
    y='term_short',
    palette='Blues_d',
    order=sig_go.sort_values('enrichment_factor', ascending=False)['term_short']
)

# 设置标题和标签
g.set_titles("{col_name}", size=14)
g.set_axis_labels("Enrichment Factor", "")
g.fig.subplots_adjust(top=0.9)
g.fig.suptitle('GO Enrichment by Category', fontsize=16)

In [ ]:
# 4. 富集网络图 (显示术语间关系)
import networkx as nx

plt.figure(figsize=(14, 12))

# 创建图对象
G = nx.Graph()

# 添加节点 (GO术语)
for _, row in sig_go.iterrows():
    G.add_node(
        row['term_short'],
        size=row['intersection_size'] * 50,
        color=row['-log10(p)'],
        category=row['category']
    )

# 添加边 (基于父术语关系)
for _, row in sig_go.iterrows():
    if isinstance(row['parents'], list):
        for parent in row['parents']:
            parent_name = sig_go[sig_go['native'] == parent]['term_short']
            if not parent_name.empty:
                G.add_edge(row['term_short'], parent_name.values[0])

# 设置节点颜色和大小
node_colors = [G.nodes[n]['color'] for n in G.nodes]
node_sizes = [G.nodes[n]['size'] for n in G.nodes]

# 绘制网络
pos = nx.spring_layout(G, k=0.5)
nx.draw_networkx_nodes(
    G, pos,
    node_size=node_sizes,
    node_color=node_colors,
    cmap=plt.cm.plasma,
    alpha=0.8
)
nx.draw_networkx_edges(G, pos, alpha=0.2)
nx.draw_networkx_labels(G, pos, font_size=10)

# 添加图例
sm = plt.cm.ScalarMappable(cmap=plt.cm.plasma, 
                          norm=plt.Normalize(vmin=min(node_colors), 
                          vmax=max(node_colors)))
sm.set_array([])
cbar = plt.colorbar(sm, shrink=0.8)
cbar.set_label('-log10(p-value)')

plt.title('GO Term Relationships Network', fontsize=16)
plt.axis('off')
plt.tight_layout()
plt.show()

In [ ]:
sns.barplot(  # 将绘图对象赋值给ax变量以便后续调整
    data=sig_go[sig_go['source']=='GO:BP'],
    y='enrichment_factor',
    x='term_short',   # 对调x和y轴
    # hue='category',
    dodge=False,
    palette='Set2'
)

In [ ]:
import matplotlib.pyplot as plt
import seaborn as sns

# 绘制条形图
plt.figure(figsize=(14, 8))
ax = sns.barplot(  # 将绘图对象赋值给ax变量以便后续调整
    data=sig_go[sig_go['source']=='GO:BP'],
    y='enrichment_factor',
    x='term_short',   # 对调x和y轴
  	# hue='category',
    dodge=False,
    palette='Set2'
)

# 设置布局和标签
plt.xlabel('Enrichment Factor', fontsize=12)
plt.ylabel('GO Terms', fontsize=12)
plt.title('GO Enrichment Analysis', fontsize=14)
plt.legend(title='Category', loc='best')  # 自动选择图例位置

# 增加可读性优化
ax.tick_params(axis='y', labelsize=7)  # 调整Y轴字体大小
ax.grid(axis='x', linestyle='--', alpha=0.7)  # 添加横向网格线

# 将X/Y轴互换 + 水平条形图
# sns.barplot(data=sig_go, y='enrichment_factor', x='term_short')
plt.xticks(rotation=45, ha='right')  # 旋转X轴标签

# 关键步骤：调整布局并保存为PDF
plt.tight_layout()  # 自动调整子图参数避免重叠
plt.savefig('micro_deg_go_enrichment.pdf', format='pdf', bbox_inches='tight', dpi=600)  # bbox_inches确保内容完整
